# CIND820 - Exploration Data Analysis  



I aim to predict the efficiency of COVID-19 vaccinates around the world using Data Classification and Clustering to analyze the efficiency of COVID-19 vaccines over the population infected and deaths reported.

First, I will explore the dataset to get insides and better understand patterns, detect error and outliers, and find relationships between variables. Then, identify key factors to determine the efficiency of COVID-19 vaccine in relation to the number of cases and deaths.


## Preparation
Describing the working dataset and any imposed constraints

This dataset is taken from Our World in Data website, officially collected by Our World in Data team. This dataset will be synced daily. For more info:
https://www.kaggle.com/datasets/caesarmario/our-world-in-data-covid19-dataset

Import the following files:
This dataset is taken from Our World in Data website, officially collected by Our World in Data team. This dataset will be synced daily:

https://covid.ourworldindata.org/data/owid-covid-data.csv

In [4]:
# Import libraries
import pandas as pd
from pandas_profiling import ProfileReport

import numpy as np